## Portfolio optimization project

This project focuses on implementing quantitative methods for portfolio optimization and financial analysis. It provides a Python-based framework for analyzing financial assets, calculating efficient portfolios, and visualizing risk-return tradeoffs. The key objectives are to construct portfolios that maximize returns for a given level of risk or minimize risk for a desired return, leveraging classical finance theories like Modern Portfolio Theory (MPT) and the Capital Asset Pricing Model (CAPM).

The project includes creating a class with the following functionalities:

**Data Retrieval**: Fetches intraday financial data for selected assets using the Alpha Vantage API.

**Risk and Return Analysis**: Computes average returns, variances, and covariance matrices for portfolio assets.

**Portfolio Optimization**: Implements methods to calculate minimum variance weights and expected returns for portfolios, including risk-free assets.

**Efficient Frontier & Capital Market Line**: Visualizes the efficient frontier and the capital market line to highlight optimal portfolios.

**CAPM Beta Calculation**: Estimates the beta of individual assets relative to a market index.
The tool is designed for investors, financial analysts, and students seeking practical insights into portfolio management and optimization techniques.

In [37]:
import requests
import pandas as pd
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [68]:
class Portfolio:
    def __init__(self, *args, interval, risk_free=0):
        """
        Initialize the Portfolio2 object with a list of stock symbols, interval for data retrieval,
        and an optional risk-free rate.
        
        Parameters:
        - args: Stock symbols to include in the portfolio.
        - interval: Time interval for stock data (e.g., '1min', '5min').
        - risk_free: The risk-free rate of return, default is 0.
        """
        self.riskfree_return = risk_free
        self.args = list(args)  # Store stock symbols as a list
        self.interval = interval  # Time interval for data retrieval
        self.dataframes = {}  # Dictionary to store DataFrames for each symbol
        self.returns = []  # List to store average returns
        self.risk = []  # List to store volatilities (variance)

        for symbol in self.args:
            # Construct the URL for the Alpha Vantage API
            url = (
                f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY"
                f"&symbol={symbol}&interval={interval}&apikey=I7JS2OQULR9YW24U"
            )
            r = requests.get(url)
            data = r.json()

            # Check if the API response contains the required data
            if f"Time Series ({interval})" in data:
                # Create a DataFrame from the retrieved data
                df = pd.DataFrame(data[f"Time Series ({interval})"]).T
                df = df.astype("float")  # Ensure numeric data types
                self.dataframes[symbol] = df

                # Calculate average returns and variance (risk)
                self.returns.append(df["2. high"].pct_change().mean())  # Average return
                self.risk.append(df["2. high"].var())  # Variance as risk
            else:
                print(f"Failed to fetch data for {symbol}. Response: {data}")
                self.dataframes[symbol] = None
                self.returns.append(None)
                self.risk.append(None)

        # Compute the covariance matrix and related attributes
        self.cov_matrix = np.asmatrix(self.cov_matrix_())
        self.O = np.asmatrix(np.ones(len(args)))  # Vector of ones
        self.cov_matrix_inv = np.linalg.inv(self.cov_matrix)  # Inverse of the covariance matrix

    def cov_matrix_(self):
        """
        Compute the covariance matrix of asset returns based on high prices.
        """
        # Filter out symbols with invalid data
        valid_dataframes = {k: v for k, v in self.dataframes.items() if v is not None}

        if len(valid_dataframes) < 2:
            print("Not enough valid data to compute covariance matrix.")
            return None

        # Extract "2. high" data and compute percent changes for covariance
        high_data = {
            symbol: df["2. high"].astype("float").pct_change().dropna()
            for symbol, df in valid_dataframes.items()
        }

        # Combine into a single DataFrame and compute covariance matrix
        combined_df = pd.DataFrame(high_data)
        return combined_df.cov()

    def Two_assets(self):
        """
        Calculate the weights and risk for a two-asset portfolio.
        """
        if len(self.risk) != 2:
            raise ValueError("Must be a two-asset portfolio")
        else:
            sigma1, sigma2 = self.risk[0], self.risk[1]
            corr = self.cov_matrix[0, 1]  # Correlation between the two assets

            # Handle different correlation cases
            if corr == 1:
                if sigma1 == sigma2:
                    w1, w2 = 0.5, 0.5
                    sigma = sigma1
                elif sigma1 < sigma2:
                    w2 = sigma1 / (sigma1 - sigma2)
                    w1 = 1 - w2
                    sigma = 0
                elif sigma1 > sigma2:
                    w1 = sigma2 / (sigma1 - sigma2)
                    w2 = 1 - w1
                    sigma = 0
            elif corr == -1:
                w2 = sigma1 / (sigma1 - sigma2)
                w1 = 1 - w2
                sigma = 0
            else:  # -1 < corr < 1
                w2 = (sigma1 * (sigma1 - corr * sigma2)) / (sigma1**2 + sigma2**2 - 2 * corr * sigma1 * sigma2)
                w1 = 1 - w2
                sigma = np.sqrt(((sigma1**2) * (sigma2**2) * (1 - corr**2)) / (sigma1**2 + sigma2**2 - 2 * corr * sigma1 * sigma2))
            return w1, w2, sigma

    def min_weight(self):
        """
        Compute the minimum variance portfolio weights.
        """
        if self.riskfree_return == 0:
            return (self.O * self.cov_matrix_inv) / (self.O * self.cov_matrix_inv * np.transpose(self.O))
        else:
            return ((self.returns - (self.riskfree_return * self.O)) * self.cov_matrix_inv) / (
                (self.returns - (self.riskfree_return * self.O)) * self.cov_matrix_inv * np.transpose(self.O)
            )

    def min_weight_return(self, exp):
        """
        Compute the portfolio weights for a given expected return.
        """
        m1, m2, m3 = np.zeros((2, 2)), np.zeros((2, 2)), np.zeros((2, 2))
        m1[0, 1] = np.asmatrix(self.returns) * self.cov_matrix_inv * np.transpose(self.O)
        m1[1, 1] = self.O * self.cov_matrix_inv * np.transpose(self.O)
        m2[0, 0] = np.asmatrix(self.returns) * self.cov_matrix_inv * np.transpose(np.asmatrix(self.returns))
        m2[1, 0] = self.O * self.cov_matrix_inv * np.transpose(np.asmatrix(self.returns))
        m1[:, 0] = [exp, 1]
        m2[:, 1] = [exp, 1]
        m3[:, 1] = m1[:, 1]
        m3[:, 0] = m2[:, 0]
        return ((np.linalg.det(m1) * np.asmatrix(self.returns) * self.cov_matrix_inv) +
                (np.linalg.det(m2) * self.O * self.cov_matrix_inv)) / (np.linalg.det(m3))

    def capital_market_line(self):
        """
        Plot the Capital Market Line and Efficient Frontier.
        """
        # Define risk and return ranges
        risk_values = np.linspace(0.001, 0.4, 1000)
        expected_values = np.linspace(0.001, 0.4, 1000)

        # Risk and return of the risky portfolio
        mu_der = self.min_return()
        sigma_der = self.min_risk()

        # Compute the frontier and capital market line
        risk_for_expected = np.array([self.min_risk_return(j) for j in expected_values])
        capital_market_line = (((mu_der - self.riskfree_return) / sigma_der) * risk_values) + self.riskfree_return

        # Create the plot
        plt.figure(figsize=(10, 6))
        plt.plot(risk_values, capital_market_line, color="blue", label="Capital Market Line")
        plt.plot(risk_for_expected, expected_values, color="green", label="Efficient Frontier")
        plt.plot(self.min_risk(), self.min_return(), '-ro', color="red")
        plt.title("Capital Market Line")
        plt.xlabel("$\sigma \;$ Risk")
        plt.ylabel("$\mu$ Return")
        plt.legend()
        plt.grid(True)
        plt.show()
        
    def min_risk(self):
        """
        Compute the minimum risk 
        """
        return float(np.sqrt(self.min_weight()*self.cov_matrix*np.asmatrix(np.transpose(self.min_weight()))))
        
    def min_return(self):
        """
        Compute minimum returns
        """
        return float(self.returns*np.transpose(self.min_weight()))
    
    def min_risk_return(self,exp):
        return float(np.sqrt(self.min_weight_return(exp) * self.cov_matrix * np.asmatrix(np.transpose(self.min_weight_return(exp)))))
        
    #def min_return_ret(self,exp):
        #return self.returns*np.transpose(self.min_weight_return(exp))

    def CAPM_beta(self, asset_ticker, market_index_ticker):
        """
        Calculate the beta of an asset using CAPM.
        """
        # Retrieve asset data
        if asset_ticker in self.args:
            self.asset_return = self.dataframes[asset_ticker]["2. high"].pct_change()
        else:
            url = (
                f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY"
                f"&symbol={asset_ticker}&interval={self.interval}&apikey=I7JS2OQULR9YW24U"
            )
            r = requests.get(url)
            data = r.json()
            self.asset_return = pd.DataFrame(data[f"Time Series {self.interval}"]).T

        # Retrieve market index data
        url = (
            f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY"
            f"&symbol={market_index_ticker}&interval={self.interval}&apikey=I7JS2OQULR9YW24U"
        )
        r = requests.get(url)
        data = r.json()
        self.market_index_returns = pd.DataFrame(data[f"Time Series {self.interval}"]).T.astype("float")["2. high"].pct_change()

        # Compute beta using regression
        self.X = np.array(self.market_index_returns - self.riskfree_return)
        self.y = self.asset_return
        self.X_with_intercept = np.hstack([np.ones((self.X.shape[0], 1)) * self.riskfree_return, np.transpose(np.asmatrix(self.X))])
        model = sm.OLS(self.y, self.X_with_intercept, missing='drop').fit()

        # Return beta values
        self.beta = model.params[1]
        return self.beta


Our Portfolio class takes into account smaller intervals (1min, 5min, 15min, 30min, 60min) than libraries like PyPortfolioOpt


We will see how the class runs using a random portfolio of three stocks (NVIDIA, Infosys, and Wheaton), with returns taken every 30 minutes, and a risk free rate 4.25% based on the 10 year U.S treasury bond.

In [39]:
pf=Portfolio("NVDA","INFY","WPM",interval="30min",risk_free=0.0425/(266*10))

In [63]:
print(f' Assets returns in order are: {pf.returns}')
print(f' Assets risks in order are: pf.returns: {pf.risk}')


 Assets returns in order are: [-0.0003256942141530918, 0.0005640447096450781, -4.6378850720844615e-05]
 Assets risks in order are: pf.returns: [4.933014571652525, 0.10237496171616171, 0.4719451959636369]


In this case, the optimal portfolio allocation suggests shorting Nvidia stock to increase long positions in Infosys and Wheaton Metals Corp

In [64]:
print(f'The minimum weights for a Capital Market Portfolio: {pf.min_weight()}')
print(f'The minimum risk is: {pf.min_risk()}')
print(f'The minimum return for a minimum risk Portfolio is: {pf.min_return()}')

The minimum weights for a Capital Market Portfolio: [[-0.1386813   1.07503462  0.06364668]]
The minimum risk is: 0.0064358876594998605
The minimum return for a minimum risk Portfolio is: 0.0006485834254211754


In [ ]:
Now let´s check the weights of a minimum risk portfolio with an expected return of 1% in this time interval

In [66]:
print(f'The weights for a Portfolio with a 1% return and minimum risk is:{pf.min_weight_return(0.01)}')
print(f'The minimum risk is: {pf.min_risk_return(0.01)}')

The weights for a Portfolio with a 1% return and minimum risk is:[[-5.17216006 14.09138776 -7.9192277 ]]
The minimum risk is: 0.12438093318663844


We can see that the minimum risk is 12% for a Portfolio with and expected return of 1%. A lot higher in comparision with the minimum risk Portfolio from before that returned a 0.6% risk

Now let´s compute the Nvidia Beta from the Capital Asset Pricing model, taking the S&P 500 (Computed through Vangard ETF) as the baseline market returns.
We can see that &\Beta$>1. Meaning a higher risk than the market but also a greater level of expected returns

In [67]:
pf.CAPM_beta("NVDA","VOO")

1.12